In [ ]:
from SRCNNs.py import *
from efficient_train.py import *
from patch_transformations.py import *
from predict.py import *
from tranformation_func.py import *


import scipy.misc
import random
import os
from glob import glob
import cv2
from skimage.transform import pyramid_reduce, rotate

In [ ]:
############## generate training data ##############
list_files_HR = glob(os.path.join('../Images/DIV2K_train_HR/', "*.png"))

hr = []
lr = []

def train_crop(img, crop_factor):
    r,c, _ = img.shape
    rcrop, ccrop = int(r/crop_factor), int(c/crop_factor)
    croped = []
    for i in range(crop_factor):
        for j in range(crop_factor):
            croped.append(img[i*rcrop:(i+1)*rcrop,j*ccrop:(j+1)*ccrop,:])
    return croped

############ choose 2 from 800
small_sample = random.sample(range(1,len(list_files_HR)), 2)

version = 1
for i in range(len(small_sample)): 
    k = small_sample[i]
    file_HR = list_files_HR[k]
    img_hr = cv2.imread(file_HR, cv2.IMREAD_COLOR)
    img_hr = cv2.cvtColor(img_hr, cv2.COLOR_BGR2YCR_CB) #cv2.COLOR_BGR2RGB
    target_size = (480, 480)  #(512, 512), 16, 2 #(512, 512), 8, 4
    crop_factor = 10
    scale = 4
    shape = img_hr.shape[:2]
    off_set_x = random.randint(0, int((shape[1] - target_size[1])/2))
    off_set_y = random.randint(0, int((shape[0] - target_size[0])/2))
    img_hr = img_hr[off_set_y:off_set_y + target_size[0], off_set_x:off_set_x + target_size[1]]
    
    # v1
    if version == 1:
        img_lr = cv2.resize(img_hr, (target_size[1] // (scale//2),
                                    target_size[0] // (scale//2)), interpolation=cv2.INTER_LINEAR)
        img_lr = cv2.GaussianBlur(img_lr,(5,5),0)
        img_lr = cv2.resize(img_lr, (target_size[1] // scale,
                                    target_size[0] // scale), interpolation=cv2.INTER_NEAREST)
        img_lr = cv2.GaussianBlur(img_lr,(3,3),0)

    # v2
    if version == 2:
        img_lr = cv2.resize(img_hr, (target_size[1] // (scale//2),
                                    target_size[0] // (scale//2)), interpolation=cv2.INTER_LANCZOS4)
        img_lr = cv2.resize(img_lr, (target_size[1] // scale,
                                    target_size[0] // scale), interpolation=cv2.INTER_NEAREST)
        img_lr = cv2.GaussianBlur(img_lr,(5,5),0)

    # v3
    if version == 3:
        img_lr = pyramid_reduce(img_hr, downscale=4, sigma=None, order=1, mode='reflect', cval=0, multichannel=True)
        img_lr = cv2.GaussianBlur(img_lr,(5,5),0)

    # v4
    if version == 4:
        img_lr = pyramid_reduce(img_hr, downscale=2, sigma=None, order=1, mode='reflect', cval=0, multichannel=True)
        img_lr = cv2.GaussianBlur(img_lr,(7,7),0)
        img_lr = pyramid_reduce(img_lr, downscale=2, sigma=None, order=1, mode='constant', cval=0, multichannel=True)
        img_lr = cv2.GaussianBlur(img_lr,(3,3),0)
    
    hr.extend(train_crop(img_hr, crop_factor))
    lr.extend(train_crop(img_lr, crop_factor))
    
idx = np.random.randint(len(hr), size=250) ##### randomly draw patches
hr = np.asarray(hr)[idx,:,:,:]
lr = np.asarray(lr)[idx,:,:,:]
    
print(len(hr), hr[0].shape)
print(len(lr), lr[0].shape)

In [ ]:
##### processing raw data
x = hr
y = lr
for i in range(len(x_data)):
    x_90 = cv2.rotate(x_data[i], cv2.ROTATE_90_CLOCKWISE)
    x_180 = cv2.rotate(x_data[i], cv2.ROTATE_180)
    x_270 = cv2.rotate(x_data[i], cv2.ROTATE_90_COUNTERCLOCKWISE)
    y_90 = cv2.rotate(y_data[i], cv2.ROTATE_90_CLOCKWISE)
    y_180 = cv2.rotate(y_data[i], cv2.ROTATE_180)
    y_270 = cv2.rotate(y_data[i], cv2.ROTATE_90_COUNTERCLOCKWISE)
    x.extend((x_data[i], x_90, x_180, x_270))
    y.extend((y_data[i], y_90, y_180, y_270))

Yx_data= np.asarray(x)[:,:,:,0]
Yy_data= np.asarray(y)[:,:,:,0]

input_r, input_c = Yx_data.shape[1], Yx_data.shape[2]

input_shape_dct, output_shape_dct, x_train_dct, y_train_dct = process_data(Yx_data,Yy_data,space) 
input_shape_db6, output_shape_db6, x_train_db6, y_train_db6 = process_data_wave(Yx_data,Yy_data,db6)

padding_shape = np.max([input_shape_db6[0], input_shape_dct[0]])
train_unpad = [x_train_dct, x_train_db6]
train = []
## pad to the same shape
for i in range(len(train_unpad)):
    if train_unpad[i].shape[1] == padding_shape:
        train.append(train_unpad[i])
    else:
        target = train_unpad[i]
        v = []
        for j in range(target.shape[0]):
            temp = padding(target[j, :,:,:], padding_shape)
            v.append(temp)
        train.append(np.asarray(v))
        
## fold to one tensor
for i in range(1,len(train)):
    train[0] = np.concatenate((train[0], train[i]), axis=3)
x_train = train[0]
y_train = np.reshape(Yy_data, (Yy_data.shape[0],Yy_data.shape[1],Yy_data.shape[2],1))
input_shape = x_train.shape[1:]
output_shape = y_train.shape[1:]
print(x_train.shape, y_train.shape)

In [ ]:
input_img = Input(shape = input_shape)
models = Model(input_img, frame_dct(input_img, output_shape))
models.compile(loss= 'mean_squared_error', optimizer = 'adam')
models.summary()

epoch_count = 0       # initial epoch setting
lower_bound = 10e4    # threshold of explosion, above this value, read and retrain
psnr_value = 0        # initial psnr value setting
marker = 500          # max loss when trying testing psnr value
filename = 'edsr_resnet.h5'
max_epoch = 1000      # max number of epoch to train
lower_bound_min = 50  # hard stop loss value of training
####models = load_model(filename,custom_objects={'tf':tf})

efficienct_train(models, x_train, y_train, epoch_count, lower_bound,marker, psnr_value,filename, max_epoch, lower_bound_min)